In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pv_1_5 as pv
import time
from pydae.plot_tools import set_style

colors = set_style(plt)

In [5]:
### Instantiation
model = pv.model()

### Initial state computation
params = {}
M  = 1
N  = 2
for i_m in range(1,M+1):
    for i_n in range(1,N+1):
        name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
        params.update({f'irrad_LV{name}':1000+(np.random.rand()-0.5)*0})
        params.update({f'p_s_ppc_LV{name}':2.0,f'q_s_ppc_LV{name}':1})
        params.update({f'N_pv_s_LV{name}':20, f'N_pv_p_LV{name}':200})

#model.report_u()
#model.report_x()
#model.report_y()
model.ini(params,'xy_0.json')


True

In [ ]:
### Time domain simulation
Δt = 0.05
model = pv_100.model()
model.Dt = 0.025
times = np.arange(0,5,Δt)
V_poi_ref_rec = np.copy(times)

params_ini = {'S_n_GRID':2e6}
for ig in range(1,1+N_gen):  # change in reactive power reference
    name = 'LV' + f'{ig}'.zfill(3)
    params_ini.update({f'q_s_ppc_{name}':0})

model.ini(params_ini,'xy_0.json')  # b_1_2 is the inverse of the line reactance (R_line = 0)

V_poi_0 = model.get_value('V_POI')
q_s_ppc = 0.0
K_vp = 0.9
K_vi = 50
xi_v = 0.0
params_ctrl = {}

for it,t in enumerate(times):

    V_poi_ref = V_poi_0
    if t>1.0:
        V_poi_ref = 1.0  # change in POI voltage reference

    model.u_run[q_s_ppc_idxs] = q_s_ppc

    # if t>3.0:
    #     for ig in range(1,1+N_gen,3):  # change in reactive power reference
    #         name = 'LV' + f'{ig}'.zfill(3)
    #         params_ctrl.update({f'temp_deg_{name}':75})

    t_0 = time.perf_counter_ns()
    model.step(t,{})
    t_1 = time.perf_counter_ns()
    
    V_poi = model.get_value('V_POI')
    epsilon_v = V_poi_ref - V_poi
    xi_v += Δt*epsilon_v 
    q_s_ppc = K_vp*epsilon_v +  K_vi*xi_v
    
    V_poi_ref_rec[it] = V_poi_ref

    print(f't = {t:4.2f} s, time = {(t_1-t_0)/1e6:4.0f} ms, V_poi = {V_poi:3.2f}')

In [10]:
model = pv.model()

M  = 1
N  = 5
model.set_value('b_POI_GRID',-0.1*M*N)
gnames = []
for i_m in range(1,M+1):
    for i_n in range(1,N+1):
        name = f"{i_m}".zfill(2) + f"{i_n}".zfill(2)
        model.set_value(f'T_lp1_LV{name}',0.01)
        model.set_value(f'p_s_ppc_LV{name}',0.1)
        model.set_value(f'q_s_ppc_LV{name}',0.2)
        model.set_value(f'irrad_LV{name}', 800+(np.random.rand()-0.5)*0)
        model.set_value(f'p_s_ppc_LV{name}',2.0)
        model.set_value(f'q_s_ppc_LV{name}',0.0)
        model.set_value(f'N_pv_s_LV{name}',20)
        model.set_value(f'N_pv_p_LV{name}',200)
        gnames += [name]

idxs_q_s_ppc = [model.u_ini_list.index(f'q_s_ppc_LV{item}')  for item in gnames]
idxs_q_s = [model.y_ini_list.index(f'q_s_LV{item}')  for item in gnames]

# first call
t_0 = time.perf_counter()
conv = model.ini({},'xy_0.json')
t_1 = time.perf_counter()
print(f'time ini first call = {t_1-t_0}')

# call with initial guesses
t_0 = time.perf_counter()
conv = model.ini({},'xy_0.json')
t_1 = time.perf_counter()
print(f'time call with initial guesses = {t_1-t_0}')

# call single iteration
t_0 = time.perf_counter()
conv = model.ini({})
t_1 = time.perf_counter()
print(f'time single iteration = {t_1-t_0}')

Dt_mid = 1

# for t in np.arange(0,10.0,Dt_mid):

#     t_end = t + Dt_mid
#     q_ppc = 1.0*np.sin(t*10)+0.5
#     model.u_ini[idxs_q_s_ppc] = q_ppc

#     t_0 = time.perf_counter()
#     conv = model.ini({},'xy_0.json')    
#     t_1 = time.perf_counter()
#     V_POI = model.xy[model.N_x+model.y_run_list.index('V_POI')] 
#     V_LV0101 = model.xy[model.N_x+model.y_run_list.index('V_LV0101')] 
#     print(f'time mkl = {(t_1-t_0)*1000:2.3f} ms, V_POI = {V_POI:6.4f}, V_LV0101 = {V_LV0101:6.4f}, its = {model.iniintparams[2]}')



### step function

#
# model.eval_preconditioner_trap()

pt = np.zeros(64,dtype=np.int32)
xy = model.xy
x = xy[:model.N_x]
y_run = xy[model.N_x:]
 
Dt_mid = 0.1
its = 0

model.Dt = 0.025

for gname in gnames:
    model.set_value(f'T_lp1_LV{gname}',0.2)
    model.set_value(f'ramp_enable_LV{gname}',1.0)

## Main simulation loop

for it,t in enumerate(np.arange(0,1.0,Dt_mid)):

    t_0 = time.perf_counter()
    t_end = t + Dt_mid
    q_ppc = -0.5*np.sin(t*5)+0.5
    model.u_run[idxs_q_s_ppc] = q_ppc

    model.step(t,{})
    
    V_POI = model.xy[model.N_x+model.y_run_list.index('V_POI')] 
    V_LV0101 = model.xy[model.N_x+model.y_run_list.index('V_LV0101')]
    q_s_LV0101 = model.xy[model.N_x+model.y_run_list.index('q_s_LV0101')] 
    t_1 = time.perf_counter()
    print(f'time mkl = {(t_1-t_0)*1000:2.3f} ms, V_POI = {V_POI:6.4f}, V_LV0101 = {V_LV0101:6.4f}, q_s_LV0101 = {q_s_LV0101:6.4f}, its = {model.runintparams[2]}')



# s = svg('pv_m_n_base.svg')

# for name in gnames:
#     V = model.xy[model.N_x+model.y_run_list.index(f'V_LV{name}')] 
#     color = np.clip(int(255*(V-1))*5,-255,255)
#     if color >0:
#         s.set_color('circle',f'C_{name}',(color,0,0))
#     if color <0:
#         s.set_color('circle',f'C_{name}',(0,0,-color))

# for name in gnames:
#     V = model.xy[model.N_x+model.y_run_list.index(f'V_MV{name}')] 
#     color = np.clip(int(255*(V-1))*10,-255,255)
#     if color >0:
#         s.set_color('path',f'B_{name}',(color,0,0))
#     if color <0:
#         s.set_color('path',f'B_{name}',(0,0,-color))

# s.save('pv_10_10.svg')
 


# for t in np.arange(0,10.0,Dt_mid):
#     t_end = t + Dt_mid
#     q_ppc = 1.0*np.sin(t*10)
#     model.u_run[idxs_q_s_ppc] = q_ppc

#     t_0 = time.perf_counter()
#     #model.step(t,{})
#     # step(         * pt,t, t_end,    *jac_trap,  *indptr, *indices, *f, *g, *fg, *x, *y,     *xy, *x_0, *f_0, *Dxy,        *u,       *p,    N_x,    N_y, max_it, itol, its, Dt);
#     solver.lib.step(p_pt,t, t_end,p_sp_jac_trap, p_indptr,p_indices,p_f,p_g,p_fg,p_x,p_y_run,p_xy,p_x_0,p_f_0, p_Dxy,  p_u_run,      p_p,    N_x,    N_y, max_it, itol, its, model.Dt)
    
#     t_1 = time.perf_counter()

#     V_POI = xy[model.N_x + model.y_run_list.index('V_POI')] 
#     V_POI_MV = xy[model.N_x + model.y_run_list.index('V_POI_MV')] 
#     #Q_POI = xy[model.N_x + model.y_run_list.index('q_line_POI_GRID')] 
#     # V_I02_1_i = xy[model.y_run_list.index('V_I02_1_i')] 
#     # V_I02_1 = V_I02_1_r + 1j*V_I02_1_i
#     # V_I02_1_m = np.abs(V_I02_1)

#     print(f't/s= {t_1-t_0:0.3f}, q_ppc = {q_ppc:7.4f}, V_POI_MV = {V_POI_MV:5.4f}, V_POI = {V_POI:5.4f}')






time ini first call = 0.007984299911186099
time call with initial guesses = 0.00784970005042851
time single iteration = 0.002400800120085478
time mkl = 0.259 ms, V_POI = 0.9913, V_LV0101 = 0.9973, q_s_LV0101 = -0.0000, its = 0
time mkl = 6.520 ms, V_POI = 0.9955, V_LV0101 = 1.0035, q_s_LV0101 = 0.0400, its = 2
time mkl = 15.154 ms, V_POI = 1.0001, V_LV0101 = 1.0104, q_s_LV0101 = 0.0854, its = 1
time mkl = 20.583 ms, V_POI = 0.9946, V_LV0101 = 1.0022, q_s_LV0101 = 0.0316, its = 1
time mkl = 18.986 ms, V_POI = 0.9953, V_LV0101 = 1.0033, q_s_LV0101 = 0.0387, its = 1
time mkl = 23.777 ms, V_POI = 1.0090, V_LV0101 = 1.0239, q_s_LV0101 = 0.1758, its = 1
time mkl = 32.152 ms, V_POI = 1.0308, V_LV0101 = 1.0567, q_s_LV0101 = 0.4060, its = 1
time mkl = 33.760 ms, V_POI = 1.0533, V_LV0101 = 1.0905, q_s_LV0101 = 0.6601, its = 1
time mkl = 42.760 ms, V_POI = 1.0709, V_LV0101 = 1.1171, q_s_LV0101 = 0.8708, its = 1
time mkl = 37.582 ms, V_POI = 1.0802, V_LV0101 = 1.1312, q_s_LV0101 = 0.9863, its = 1
